In [1]:
import os
os.chdir('../../vlm_toolbox/')

/home/alireza/vlm_toolbox


In [3]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [4]:
import gc
import warnings

import torch

from config.enums import (
    ImageDatasets,
    ModelType,
    Setups,
    Sources,
    Stages,
    Trainers,
)
from config.setup import Setup
from pipeline.pipeline import Pipeline

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')

In [5]:
def flush():
    gc.collect()
    torch.cuda.empty_cache()

# Config

In [8]:
HIERARCHY_LEVELS = {
    'coarse': None,
    'default': 'coarse',
}

REVERSED_HIERARCHY_LEVELS = {
    'coarse': 'default',
    'default': None,
}

LEVELS_TOP_N_CHILDREN_TO_PROPAGATE = {
    'coarse': 3,
    'default': None,
}

TOP_K = 5
LEVEL_NAMES = list(HIERARCHY_LEVELS.keys())

In [44]:
for col_name in cols[-1:]:
    train_stage_setup = Setup(
        dataset_name=ImageDatasets.IMAGENET_1K,
        trainer_name=Trainers.COOP,
        setup_type=Setups.EVAL_ONLY,
        model_type=ModelType.ZERO_SHOT,
        source=Sources.OPEN_AI,
        train_split='validation',
        label_column_name=col_name,
        n_shots=16,
        # annotations_key_value_criteria={'kingdom': ['Animalia']},
        top_k=len(labels_df[col_name].value_counts()),
        load_from_checkpoint=True,
    )

    # print(train_stage_setup.get_model_checkpoint_path())

    pipeline = Pipeline(train_stage_setup, device_type=DEVICE_TYPE, logger=logger)
    pipeline.setup_labels()
    pipeline.setup_model()
    pipeline.setup_data(stages=[Stages.EVAL])
    metrics = pipeline.evaluate()
    print(metrics)
    to_save_dir = '/home/alireza/novel_coop/' + train_stage_setup.get_relative_save_path()
    saved_dirs_dict = pipeline.save_metrics(directory=to_save_dir, main_metric_only=False)
    print(saved_dirs_dict)
    pipeline.tear_down()
    del pipeline
    del train_stage_setup
    flush()

Model weights loaded successfully from /home/alireza/io/model/zero_shot/imagenet1k/clip_vit_b_16/open_ai/coop/16_shots/default/pytorch_model.bin


{'eval_loss': 1.1691420078277588, 'eval_top_1': 0.69364, 'eval_top_2': 0.81794, 'eval_top_3': 0.86794, 'eval_top_4': 0.8945, 'eval_top_5': 0.91232, 'eval_runtime': 102.4459, 'eval_samples_per_second': 488.063, 'eval_steps_per_second': 0.478}
{'per_sample': '/home/alireza/novel_coop/zero_shot/imagenet1k/clip_vit_b_16/open_ai/coop/16_shots/default/per_sample.parquet', 'overall': '/home/alireza/novel_coop/zero_shot/imagenet1k/clip_vit_b_16/open_ai/coop/16_shots/default/overall.parquet'}
